In [3]:
import urllib  
import requests
from urllib.request import urlopen 
import bs4
import json
import numpy as np
import pandas as pd
from pandas import DataFrame

# 应用Google api进行地址经纬度转换
def getGeoForAddress(address):  
    # address = "中文地址"  
    addressUrl = "http://maps.googleapis.com/maps/api/geocode/json?address=" + address  
    #中文url需要转码才能识别  
    addressUrlQuote = urllib.parse.quote(addressUrl, safe=':?=/')  
    response = urlopen(addressUrlQuote).read().decode('utf-8')  
    responseJson = json.loads(response)  
    #type of response is string  
    # print(type(response))  
    #type of responseJson is dict  
    # print(type(responseJson))  
    lat = responseJson.get('results')[0]['geometry']['location']['lat']  
    lng = responseJson.get('results')[0]['geometry']['location']['lng']  
    return [lat, lng]  
# 从沃尔玛中国获取网站资料  
url = 'http://www.wal-martchina.com/walmart/wminchina.htm'  
response = urlopen(url)
soup = bs4.BeautifulSoup(response, 'html.parser')
store=''
address=''
tel=''
# 分别存储店址店名，经纬度的字符串
out_list = []
ll_list = []

# 获取店址店名的数据
for table in soup.findAll('table', width="780"):

    for row in table.findAll('tr')[1:]:
        cells=row.findAll('td')
        if len(cells)==4:
                store = cells[1].find(text=True)
                address = cells[2].find(text=True)
                tel = cells[3].find(text=True)
                out_list.append([store, address])
                
        elif len(cells)==5:
                store = cells[2].find(text=True)
                address = cells[3].find(text=True)
                tel = cells[4].find(text=True)
                out_list.append([store, address])
                
        elif len(cells)==3:
                store = cells[0].find(text=True)
                address = cells[1].find(text=True)
                tel = cells[2].find(text=True)
                out_list.append([store, address])
# 根据店名，店址计算经纬度，先用店名再用店址，两者都实现不了输出 No Results
for i in range(0,len(out_list)):
    try:
        lat = getGeoForAddress(out_list[i][0])[0]
        lng = getGeoForAddress(out_list[i][0])[1]
    except ValueError:
        lat= getGeoForAddress(out_list[i][1])[0]
        lng = getGeoForAddress(out_list[i][1])[1]
    except:
        lat = "No Results"
        lng = "No Results"
    ll_list.append([lat,lng])

# 将此前连个list合并并转化成array
final_list = [x+y for x,y in zip(out_list, ll_list)]
final_array = np.asarray(final_list)

# 将array转化成dataframe存储
df = DataFrame({"店名":final_array[:,0],"店址":final_array[:,1],
                "纬度":final_array[:,2],"经度":final_array[:,3]})

# 建立xlsx文件
writer = pd.ExcelWriter('沃尔玛中国店址及经纬度.xlsx', engine='xlsxwriter')

# 将dataframe写入已建好的Excel文件
df.to_excel(writer, sheet_name='Sheet1',index=False)
# 用户可以直接编辑使用Excel文件，检索所需内容